In [1]:
#Registering new person
import os
from audio_utils.recorder import Recorder

base_dir = '/run/media/hbritto/Data/Datasets/deep-speaker-data'
# base_dir = '/home/hbritto/code/tcc/deep-speaker-data'
audio_dir = base_dir + '/VCTK-Corpus/'
cache_dir = base_dir + '/cache/'

In [2]:
import pickle
from deep_speaker.audio_reader import AudioReader
from deep_speaker.constants import c
with open(os.path.join(cache_dir, 'embeddings.pkl'), 'rb') as pkl:
    all_embs = pickle.load(pkl)

In [ ]:
name = input('Nome da pessoa a ser registrada: ')
n_audio = int(input('Número de áudios a serem gravados: '))
person_folder = os.path.join(base_dir, name)
os.makedirs(person_folder, exist_ok=True)
rec = Recorder()

for i in range(n_audio):
    print('Áudio número {:>02d} de {:>02d}'.format(i + 1, n_audio))
    with rec.open(os.path.join(person_folder, '{}_{:>03d}.wav'.format(name, i))) as recfile:
        recfile.record(8)

In [3]:
# new_audio_dir = os.path.join(base_dir, person_folder, '')
audio_reader = AudioReader(input_audio_dir=audio_dir,
                           output_cache_dir=cache_dir,
                           sample_rate=c.AUDIO.SAMPLE_RATE,
                           multi_threading=True)
audio_reader.build_cache()

In [4]:
from deep_speaker.unseen_speakers import inference_embeddings
emb_teste = inference_embeddings(audio_reader, 'p225')

Using TensorFlow backend.


Embeddings will be normalized.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 390)               0         
_________________________________________________________________
fc1 (Dense)                  (None, 200)               78200     
_________________________________________________________________
normalization (Lambda)       (None, 200)               0         
_________________________________________________________________
embeddings (Lambda)          (None, 200)               0         
_________________________________________________________________
softmax (Dense)              (None, 105)               21105     
Total params: 99,305
Trainable params: 21,105
Non-trainable params: 78,200
_________________________________________________________________
None
********************************************************************************
[0.07174713 0.05

In [ ]:
# all_embs.update({'Teste': emb_teste})
# del all_embs['Teste']
print(emb_teste)
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(n_neighbors=1, metric='cosine')
nn.fit(list(all_embs.values()))

In [ ]:
nn.kneighbors(emb_teste.reshape(1, -1))

In [5]:
from blocks.voice_recogniser import VoiceRecogniser
recog = VoiceRecogniser()
recog.persons = all_embs

TypeError: fit() takes 3 positional arguments but 111 were given

In [ ]:
recog.recognise(emb_teste)